In [1]:
import json

with open("../data/AIC23_Track2_NL_Retrieval/data/test-tracks.json") as f:
    test_tracks = json.load(f)

with open("../data/AIC23_Track2_NL_Retrieval/data/test-queries.json") as f:
    test_queries = json.load(f)

In [2]:
test_queries["8d356ab3-2427-488c-8f55-c1aa27bb6c7f"]

{'nl': ['A white sedan turns right at an intersection following another red sedan.',
  'A white sedan turns right.',
  'A white sedan turns right after another car.'],
 'nl_other_views': ['A gray sedan crosses the intersection keeping straight.',
  'A gray sedan keeps straight.',
  'A gray sedan going straight down the street.',
  'A gray sedan keeping straight on the right side of the street.',
  'A silver sedan.',
  'A silver sedan runs down the street.']}

In [3]:
track = test_tracks["4cf64bda-3fb7-4d74-b28f-ce4d7e20865b"]

KeyError: '4cf64bda-3fb7-4d74-b28f-ce4d7e20865b'

In [4]:
{
    1: 'straight',
    2: 'stop',
    3: 'left',
    4: 'right'
}

{1: 'straight', 2: 'stop', 3: 'left', 4: 'right'}

In [50]:
import os
import matplotlib.pyplot as plt
from PIL import Image

f, axarr = plt.subplots(2,2)
axarr = axarr.flatten()



data_path = "../data/AIC23_Track2_NL_Retrieval/data/"
for frame, ax in zip(track['frames'][:4], axarr):
    image = Image.open(os.path.join(data_path, frame[2:]))
    ax.imshow(image)

plt.show()

SyntaxError: invalid syntax (108610378.py, line 8)

# Generate Pseudo Labels

In [68]:
from tqdm import tqdm
import torch
import numpy as np
import spacy

clip_feats_mat = torch.load("../data/AIC23_Track2_NL_Retrieval/data/clip_feats_mat.pth").detach().cpu().numpy().T
nlp = spacy.load("en_core_web_sm")

with open("../post_process/target_test_direction_predict_one_hot_refinement.json") as f:
    test_direction = json.load(f)

loc_dict = {'c005': 1, 'c014': 0, 'c029': 0, 'c003': 1, 'c017': 0, 'c035': 0, 'c013': 0, 'c027': 0, 'c038': 0,
                    'c016': 0, 'c002': 1, 'c021': 0, 'c019': 0, 'c030': 0, 'c020': 0, 'c026': 0, 'c010': 0, 'c004': 1,
                    'c033': 0, 'c012': 1, 'c001': 1, 'c034': 0, 'c022': 0, 'c036': 0, 'c040': 1, 'c025': 0, 'c032': 0,
                    'c037': 1}

for idx, (key, val) in enumerate(tqdm(test_tracks.items())):
    text_id = np.argmax(clip_feats_mat[idx])
    text = test_queries[list(test_queries.keys())[text_id]]['nl'][0]
    
    doc = nlp(text)
    for chunk in doc.noun_chunks:
        name_object = chunk.text
        break
    
    test_tracks[key]['nl'] = []

    for i in range(3):
        new_text = f"A {name_object.lower()}"

        direction_pred = test_direction[key]
        if direction_pred[0] == 1:
            straight_opts = ["drives forward", "moving straight", "drives straight", "drive downs", "run downs"]
            straight_text = straight_opts[np.random.randint(0, len(straight_opts))]
            new_text += f" {straight_text}"
        elif direction_pred[1] == 1:
            stop_opts = ["park", "stops", "is stopping"]
            stop_text = stop_opts[np.random.randint(0, len(stop_opts))]
            new_text += f" {stop_text}"

        elif direction_pred[2] == 1:
            left_opts = ["moves left", "turns left", "keeps left"]
            left_text = left_opts[np.random.randint(0, len(left_opts))]

            new_text += f" {left_text}"
        elif direction_pred[3] == 1:
            right_opts = ["moves right", "turns right", "keeps right"]
            right_text = right_opts[np.random.randint(0, len(right_opts))]

            new_text += f" {right_text}"

        cam_id = val['frames'][0].split("/")[3]
        if loc_dict[cam_id] == 1:
            new_text += " at an intersection."
        else:
            new_text += "."
    
        test_tracks[key]['nl'].append(new_text)
        
    test_tracks[key]['nl_other_views'] = []

100%|██████████| 184/184 [00:00<00:00, 459.13it/s]


In [ ]:
test_tracks['18e09c74-6a9e-4432-9a2a-b28c85a34285']

In [70]:
# save pseudo_label files:
with open("../data/AIC23_Track2_NL_Retrieval/data/pseudo_test_tracks.json", 'w') as f:
    json.dump(test_tracks, f)

# Generate 2

In [4]:
!pip install sentence-splitter SentencePiece
!pip install sacremoses

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 880.6/880.6 kB 3.3 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
  Created wheel for sacremoses: filename=sacremoses-0.0.53-py3-none-any.whl size=895241 sha256=b09fff0419397f36b67490e6fe7e8317cdf0af4e1daa8e209949461e03209dbf
  Stored in directory: /home/aivn2023/.cache/pip/wheels/12/1c/3d/46cf06718d63a32ff798a89594b61e7f345ab6b36d909ce033
Successfully built sacremoses


In [2]:
from transformers import PegasusForConditionalGeneration, PegasusTokenizer

model_name = "tuner007/pegasus_paraphrase"
torch_device = "cpu"
tokenizer = PegasusTokenizer.from_pretrained(model_name)
model = PegasusForConditionalGeneration.from_pretrained(model_name).to(torch_device)

/home/aivn2023/anaconda3/envs/pytorch_1121_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
def paraphrase_aug(
    input_text, num_return_sequences=1, num_beams=1, max_length=100, temperature=0
):
    batch = tokenizer(
        [input_text],
        truncation=True,
        padding="longest",
        max_length=max_length,
        return_tensors="pt",
    ).to(torch_device)
    translated = model.generate(
        **batch,
        max_length=max_length,
        num_beams=num_beams,
        num_return_sequences=num_return_sequences,
        temperature=temperature
    )
    tgt_text = tokenizer.batch_decode(translated, skip_special_tokens=True)
    return tgt_text

In [4]:
import json
import os 
import sys
import uuid
import cv2
os.chdir(os.path.dirname('/media/aivn2023/86c50d28-d521-419b-a569-3aab9993961f/media/ai2023/HungAn/Track2-Vehicle-Retrieval/Track2-Vehicle_Retrieval/configs'))


with open("data/AIC23_Track2_NL_Retrieval/data/test-tracks.json") as f:
    test_tracks = json.load(f)

with open("data/AIC23_Track2_NL_Retrieval/data/test-queries.json") as f:
    test_queries = json.load(f)

with open('submissions/submit_10_45ath.json') as f:
    test_results = json.load(f)

In [6]:
# from preprocessing.transforms import build_transforms, build_vanilla_transforms, build_motion_transform, BackTranslateAug
# trans_aug = BackTranslateAug()

In [5]:
pseudo_labels = {}

for i, (key, val) in enumerate(test_results.items()):
    print(40*"-")
    query = test_queries[key]
    nls = query['nl']
    print(nls)
    texts_aug = [paraphrase_aug(nl) for nl in nls]
    print(texts_aug)
    
    # for i in range(3):
    #     track_id = val[i]
    #     track = test_tracks[track_id]
    #     id = uuid.uuid1().__str__()

    #     pseudo_labels[id] = {
    #         'frames': track['frames'],
    #         'boxes': track['boxes'],
    #         'nl': query['nl'],
    #         'nl_other_views': query['nl_other_views']
    #     }
    #     print(track_id)
    #     motion_line_path = f"../data/AIC23_Track2_NL_Retrieval/data/motion_map/{track_id}.jpg"
    #     save_motion_path = f"../data/AIC23_Track2_NL_Retrieval/data/motion_map/{id}.jpg"
    #     motion_img = cv2.imread(motion_line_path)
    #     cv2.imwrite(save_motion_path, motion_img)

----------------------------------------
['A blue pickup truck keeps straight at an intersection.', 'A blue pickup drives through the intersection.', 'A blue Pickup Truck running down the street.']
[['A blue pickup truck is driving.'], ['A blue pickup is driving.'], ['A blue pickup truck is outside.']]
----------------------------------------
['A large dark gray pickup crosses an intersection.', 'A pickup leads others.', 'A grey pickup truck runs down the road alongside a truck.']
[['A pickup crosses an intersection.'], ['A pickup is driving.'], ['A grey pickup truck is driving down the road.']]
----------------------------------------
['A blue sedan drives through an intersection.', 'A blue sedan moves straight passing cargo truck on the intersection.', 'A blue sedan runs through an intersection.']
[['A blue sedan is driving.'], ['A blue sedan is passing a cargo truck.'], ['A blue sedan is driving.']]
----------------------------------------
['A big green cargo truck drives down an in

KeyboardInterrupt: 

In [87]:
with open("../data/AIC23_Track2_NL_Retrieval/data/train-tracks.json") as f:
    train_tracks = json.load(f)

In [88]:
for key, val in train_tracks.items():
    pseudo_labels[key] = val

In [89]:
len(pseudo_labels.keys())

2707

In [91]:
# save pseudo_label files:
with open("../data/AIC23_Track2_NL_Retrieval/data/pseudo_test_tracks.json", 'w') as f:
    json.dump(pseudo_labels, f)